# Лабораторная работа 6

Задание 1 . Провести классификацию найденного датасета, методами CatBoost . В формате *Markdown* написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [1]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('anime-dataset-2023.csv')
data=data.head(500)

In [3]:
# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 500
Количество столбцов: 24

Типы данных по столбцам:
anime_id         int64
Name            object
English name    object
Other name      object
Score           object
Genres          object
Synopsis        object
Type            object
Episodes        object
Aired           object
Premiered       object
Status          object
Producers       object
Licensors       object
Studios         object
Source          object
Duration        object
Rating          object
Rank            object
Popularity       int64
Favorites        int64
Scored By       object
Members          int64
Image URL       object
dtype: object


In [4]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# Предположим, что мы хотим предсказать страну песни
X = data.drop('Type', axis=1)
y = data['Type']
X = pd.get_dummies(X)

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Метод CatBoost

Гиперпараметры:
   - depth: Этот параметр определяет глубину дерева. Увеличение значения этого параметра увеличивает сложность модели, что может привести к более точной подгонке к тренировочным данным, но также увеличивает риск переобучения.
   - learning_rate: Определяет скорость обучения модели. Этот параметр контролирует величину, на которую обновляются веса модели при каждой итерации. Более низкие значения могут способствовать стабильному обучению, но могут потребовать большего количества итераций для достижения оптимальной точности.
   - l2_leaf_reg: Этот параметр отвечает за коэффициент регуляризации L2 (Ridge). Он контролирует сложность модели, ограничивая веса и предотвращая переобучение. Большие значения могут уменьшить сложность модели, но могут также уменьшить её точность на тренировочных данных.

In [5]:
# Определение параметров для CatBoost
param_grid_catboost = {
    'depth': [1, 4, 7, 10],
    'learning_rate': [0.01, 0.1, 1],
    'l2_leaf_reg': [1, 3, 5, 9],
    'iterations': [100, 200,],
    'depth': [0, 3, 6],
    'loss_function': ['MultiClass', 'Logloss']
}

In [6]:
# Создание модели CatBoost
catboost = CatBoostClassifier()

# Подбор оптимальных параметров с помощью перекрестной проверки для CatBoost
grid_search_catboost = GridSearchCV(catboost, param_grid_catboost, refit=True, verbose=3, cv=5)
grid_search_catboost.fit(X_train_scaled, y_train)

# Получение лучших параметров для CatBoost
best_params_catboost = grid_search_catboost.best_params_
best_score_catboost = grid_search_catboost.best_score_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
0:	learn: 1.3765427	total: 162ms	remaining: 16s
1:	learn: 1.3670667	total: 162ms	remaining: 7.95s
2:	learn: 1.3578546	total: 162ms	remaining: 5.25s
3:	learn: 1.3488955	total: 163ms	remaining: 3.9s
4:	learn: 1.3401792	total: 163ms	remaining: 3.09s
5:	learn: 1.3316961	total: 163ms	remaining: 2.55s
6:	learn: 1.3234369	total: 163ms	remaining: 2.16s
7:	learn: 1.3153931	total: 163ms	remaining: 1.87s
8:	learn: 1.3075564	total: 163ms	remaining: 1.65s
9:	learn: 1.2999192	total: 163ms	remaining: 1.47s
10:	learn: 1.2924742	total: 163ms	remaining: 1.32s
11:	learn: 1.2852143	total: 163ms	remaining: 1.2s
12:	learn: 1.2781331	total: 163ms	remaining: 1.09s
13:	learn: 1.2712242	total: 164ms	remaining: 1s
14:	learn: 1.2644817	total: 164ms	remaining: 928ms
15:	learn: 1.2578999	total: 164ms	remaining: 860ms
16:	learn: 1.2514735	total: 164ms	remaining: 800ms
17:	learn: 1.2451972	total: 164ms	remaining: 747ms
18:	learn: 1.2390662	total: 164ms	re

[CV 2/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=MultiClass;, score=0.675 total time=   0.7s
0:	learn: 1.3764084	total: 160us	remaining: 15.9ms
1:	learn: 1.3668028	total: 419us	remaining: 20.5ms
2:	learn: 1.3574658	total: 528us	remaining: 17.1ms
3:	learn: 1.3483861	total: 643us	remaining: 15.5ms
4:	learn: 1.3395533	total: 757us	remaining: 14.4ms
5:	learn: 1.3309575	total: 868us	remaining: 13.6ms
6:	learn: 1.3225895	total: 978us	remaining: 13ms
7:	learn: 1.3144402	total: 1.09ms	remaining: 12.5ms
8:	learn: 1.3065016	total: 1.23ms	remaining: 12.4ms
9:	learn: 1.2987656	total: 1.34ms	remaining: 12.1ms
10:	learn: 1.2912247	total: 1.46ms	remaining: 11.9ms
11:	learn: 1.2838720	total: 1.57ms	remaining: 11.6ms
12:	learn: 1.2767007	total: 1.69ms	remaining: 11.3ms
13:	learn: 1.2697044	total: 1.8ms	remaining: 11.1ms
14:	learn: 1.2628770	total: 1.91ms	remaining: 10.8ms
15:	learn: 1.2562128	total: 2.02ms	remaining: 10.6ms
16:	learn: 1.2497063	total: 2.13ms	remain

[CV 4/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.01, loss_function=MultiClass;, score=0.662 total time=   0.7s
0:	learn: 1.3764084	total: 169us	remaining: 16.8ms
1:	learn: 1.3668028	total: 462us	remaining: 22.7ms
2:	learn: 1.3574658	total: 596us	remaining: 19.3ms
3:	learn: 1.3483861	total: 709us	remaining: 17ms
4:	learn: 1.3395533	total: 856us	remaining: 16.3ms
5:	learn: 1.3309575	total: 976us	remaining: 15.3ms
6:	learn: 1.3225895	total: 1.09ms	remaining: 14.5ms
7:	learn: 1.3144402	total: 1.2ms	remaining: 13.8ms
8:	learn: 1.3065016	total: 1.31ms	remaining: 13.2ms
9:	learn: 1.2987656	total: 1.42ms	remaining: 12.8ms
10:	learn: 1.2912247	total: 1.53ms	remaining: 12.4ms
11:	learn: 1.2838720	total: 1.65ms	remaining: 12.1ms
12:	learn: 1.2767007	total: 1.76ms	remaining: 11.8ms
13:	learn: 1.2697044	total: 1.87ms	remaining: 11.5ms
14:	learn: 1.2628770	total: 1.98ms	remaining: 11.2ms
15:	learn: 1.2562128	total: 2.09ms	remaining: 11ms
16:	learn: 1.2497063	total: 2.2ms	remainin

[CV 1/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.1, loss_function=MultiClass;, score=0.675 total time=   0.7s
0:	learn: 1.2932730	total: 155us	remaining: 15.4ms
1:	learn: 1.2239018	total: 417us	remaining: 20.5ms
2:	learn: 1.1701278	total: 531us	remaining: 17.2ms
3:	learn: 1.1273565	total: 641us	remaining: 15.4ms
4:	learn: 1.0927061	total: 754us	remaining: 14.3ms
5:	learn: 1.0642471	total: 868us	remaining: 13.6ms
6:	learn: 1.0406248	total: 979us	remaining: 13ms
7:	learn: 1.0208526	total: 1.09ms	remaining: 12.5ms
8:	learn: 1.0041915	total: 1.21ms	remaining: 12.3ms
9:	learn: 0.9900750	total: 1.32ms	remaining: 11.9ms
10:	learn: 0.9780608	total: 1.44ms	remaining: 11.6ms
11:	learn: 0.9677979	total: 1.55ms	remaining: 11.4ms
12:	learn: 0.9590043	total: 1.66ms	remaining: 11.1ms
13:	learn: 0.9514509	total: 1.78ms	remaining: 10.9ms
14:	learn: 0.9449496	total: 1.89ms	remaining: 10.7ms
15:	learn: 0.9393448	total: 2ms	remaining: 10.5ms
16:	learn: 0.9345068	total: 2.11ms	remaining

[CV 3/5] END depth=0, iterations=100, l2_leaf_reg=1, learning_rate=0.1, loss_function=MultiClass;, score=0.662 total time=   0.7s
0:	learn: 1.2919930	total: 153us	remaining: 15.2ms
1:	learn: 1.2217432	total: 412us	remaining: 20.2ms
2:	learn: 1.1673250	total: 535us	remaining: 17.3ms
3:	learn: 1.1240624	total: 651us	remaining: 15.6ms
4:	learn: 1.0890272	total: 767us	remaining: 14.6ms
5:	learn: 1.0602616	total: 872us	remaining: 13.7ms
6:	learn: 1.0363920	total: 981us	remaining: 13ms
7:	learn: 1.0164188	total: 1.1ms	remaining: 12.6ms
8:	learn: 0.9995933	total: 1.21ms	remaining: 12.2ms
9:	learn: 0.9853419	total: 1.32ms	remaining: 11.9ms
10:	learn: 0.9732166	total: 1.43ms	remaining: 11.6ms
11:	learn: 0.9628622	total: 1.54ms	remaining: 11.3ms
12:	learn: 0.9539932	total: 1.65ms	remaining: 11.1ms
13:	learn: 0.9463776	total: 1.76ms	remaining: 10.8ms
14:	learn: 0.9398251	total: 1.87ms	remaining: 10.6ms
15:	learn: 0.9341781	total: 1.98ms	remaining: 10.4ms
16:	learn: 0.9293054	total: 2.1ms	remainin

...

44:	learn: 0.0412137	total: 666ms	remaining: 2.29s
45:	learn: 0.0407833	total: 682ms	remaining: 2.28s
46:	learn: 0.0402407	total: 699ms	remaining: 2.28s
47:	learn: 0.0395072	total: 716ms	remaining: 2.27s
48:	learn: 0.0388609	total: 732ms	remaining: 2.25s
49:	learn: 0.0383155	total: 747ms	remaining: 2.24s
50:	learn: 0.0379728	total: 762ms	remaining: 2.23s
51:	learn: 0.0375694	total: 777ms	remaining: 2.21s
52:	learn: 0.0371342	total: 791ms	remaining: 2.19s
53:	learn: 0.0367615	total: 808ms	remaining: 2.19s
54:	learn: 0.0363622	total: 834ms	remaining: 2.2s
55:	learn: 0.0360342	total: 852ms	remaining: 2.19s
56:	learn: 0.0320924	total: 866ms	remaining: 2.17s
57:	learn: 0.0317749	total: 881ms	remaining: 2.16s
58:	learn: 0.0313168	total: 896ms	remaining: 2.14s
59:	learn: 0.0310036	total: 911ms	remaining: 2.13s
60:	learn: 0.0307187	total: 925ms	remaining: 2.11s
61:	learn: 0.0303678	total: 938ms	remaining: 2.09s
62:	learn: 0.0301253	total: 952ms	remaining: 2.07s
63:	learn: 0.0298789	total: 967m

15:	learn: 0.0898711	total: 232ms	remaining: 2.67s
16:	learn: 0.0884977	total: 251ms	remaining: 2.7s
17:	learn: 0.0872347	total: 265ms	remaining: 2.68s
18:	learn: 0.0856732	total: 281ms	remaining: 2.67s
19:	learn: 0.0844170	total: 301ms	remaining: 2.71s
20:	learn: 0.0830351	total: 316ms	remaining: 2.69s
21:	learn: 0.0720877	total: 332ms	remaining: 2.68s
22:	learn: 0.0711196	total: 347ms	remaining: 2.67s
23:	learn: 0.0700817	total: 363ms	remaining: 2.66s
24:	learn: 0.0687697	total: 378ms	remaining: 2.65s
25:	learn: 0.0680978	total: 393ms	remaining: 2.63s
26:	learn: 0.0672475	total: 407ms	remaining: 2.61s
27:	learn: 0.0637181	total: 423ms	remaining: 2.6s
28:	learn: 0.0627853	total: 437ms	remaining: 2.58s
29:	learn: 0.0619105	total: 453ms	remaining: 2.57s
30:	learn: 0.0612338	total: 467ms	remaining: 2.55s
31:	learn: 0.0601948	total: 482ms	remaining: 2.53s
32:	learn: 0.0594307	total: 496ms	remaining: 2.51s
33:	learn: 0.0585069	total: 511ms	remaining: 2.5s
34:	learn: 0.0579594	total: 525ms	

177:	learn: 0.0101950	total: 2.61s	remaining: 323ms
178:	learn: 0.0101630	total: 2.63s	remaining: 308ms
179:	learn: 0.0098164	total: 2.64s	remaining: 293ms
180:	learn: 0.0096079	total: 2.65s	remaining: 279ms
181:	learn: 0.0095778	total: 2.67s	remaining: 264ms
182:	learn: 0.0095553	total: 2.68s	remaining: 249ms
183:	learn: 0.0095420	total: 2.7s	remaining: 235ms
184:	learn: 0.0095223	total: 2.71s	remaining: 220ms
185:	learn: 0.0095017	total: 2.73s	remaining: 205ms
186:	learn: 0.0094786	total: 2.74s	remaining: 191ms
187:	learn: 0.0094580	total: 2.76s	remaining: 176ms
188:	learn: 0.0094397	total: 2.77s	remaining: 162ms
189:	learn: 0.0092682	total: 2.79s	remaining: 147ms
190:	learn: 0.0092496	total: 2.81s	remaining: 132ms
191:	learn: 0.0092326	total: 2.82s	remaining: 118ms
192:	learn: 0.0091857	total: 2.84s	remaining: 103ms
193:	learn: 0.0091660	total: 2.85s	remaining: 88.2ms
194:	learn: 0.0088469	total: 2.87s	remaining: 73.5ms
195:	learn: 0.0088220	total: 2.88s	remaining: 58.8ms
196:	learn

C:\Users\ilyap\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
360 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ilyap\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ilyap\anaconda3\lib\site-packages\catboost\core.py", line 5100, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "C:\Users\ilyap\anaconda3\lib\site-packages\catboost\core.py", line 2319

0:	learn: 0.3860602	total: 18.4ms	remaining: 1.82s
1:	learn: 0.2184708	total: 39.6ms	remaining: 1.94s
2:	learn: 0.1806432	total: 63.9ms	remaining: 2.07s
3:	learn: 0.1473139	total: 85.9ms	remaining: 2.06s
4:	learn: 0.1303243	total: 108ms	remaining: 2.04s
5:	learn: 0.1278543	total: 128ms	remaining: 2.01s
6:	learn: 0.1260896	total: 151ms	remaining: 2.01s
7:	learn: 0.1243976	total: 175ms	remaining: 2.01s
8:	learn: 0.1209576	total: 197ms	remaining: 2s
9:	learn: 0.1180966	total: 225ms	remaining: 2.02s
10:	learn: 0.1141064	total: 251ms	remaining: 2.03s
11:	learn: 0.1123309	total: 274ms	remaining: 2s
12:	learn: 0.1102498	total: 297ms	remaining: 1.99s
13:	learn: 0.1088513	total: 318ms	remaining: 1.95s
14:	learn: 0.1070615	total: 338ms	remaining: 1.91s
15:	learn: 0.1044841	total: 358ms	remaining: 1.88s
16:	learn: 0.1028903	total: 378ms	remaining: 1.85s
17:	learn: 0.1013990	total: 398ms	remaining: 1.81s
18:	learn: 0.0896879	total: 418ms	remaining: 1.78s
19:	learn: 0.0882494	total: 439ms	remaining

In [7]:
test_score = grid_search_catboost.score(X_test_scaled, y_test)
print("Лучшие параметры для CatBoost:", best_params_catboost)
print("Лучший результат (точность) для CatBoost:", best_score_catboost)
print("Точность на тестовом наборе данных:", test_score)

best_model = grid_search_catboost.best_estimator_
predictions = best_model.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для CatBoost: {'depth': 6, 'iterations': 100, 'l2_leaf_reg': 9, 'learning_rate': 1, 'loss_function': 'MultiClass'}
Лучший результат (точность) для CatBoost: 0.9625
Точность на тестовом наборе данных: 0.95

Отчет по классификации:
              precision    recall  f1-score   support

       Movie       0.74      1.00      0.85        14
         OVA       1.00      0.82      0.90        17
     Special       0.00      0.00      0.00         2
          TV       1.00      1.00      1.00        67

    accuracy                           0.95       100
   macro avg       0.68      0.71      0.69       100
weighted avg       0.94      0.95      0.94       100



C:\Users\ilyap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ilyap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ilyap\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
